# **input**

In [3]:
input_video="/content/test_lie (1).mp4"

# **import**

In [1]:
# !pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 15.6 MB/s eta 0:00:00


In [2]:
import cv2
import os
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from moviepy import VideoFileClip
import librosa
from joblib import load
from flask import Flask, request, jsonify


  if event.key is 'enter':



# **constrains**

In [4]:
# check if the input video has no person or has multiple persons
# returns True if the input video is correct , false otherwise
def input_validation(input_video):
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()

    cap = cv2.VideoCapture(input_video)

    if not cap.isOpened():
        print("Could not open video")

    valid = True
    while cap.isOpened():
        ret,frame = cap.read()

        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)
        if not results.detections or len(results.detections) > 1:
            valid = False
            break

    cap.release()

    return valid

In [5]:
def check_audio_in_video(video_file):
    """Check if the video contains an audio track."""
    video = VideoFileClip(video_file)
    if video.audio is None:
        return 0
    return 1

# **models**

In [ ]:
#  saved_model=load_model("/content/facial_model.h5")

In [ ]:
# model=load_model("/content/audio_model.h5")

In [ ]:
# saved_model = load_model("C:\\Users\\MSIQ\\Downloads\\deployment\\facial_model.h5")

In [35]:
# saved_model=load_model("/content/lstm_without_masking(94).h5")

In [ ]:
saved_model= r"C:\Users\MSIQ\Downloads\deployment\lstm_without_masking(94).h5"
# "C:\Users\MSIQ\Downloads\deployment\lstm_without_masking(94).h5"

In [ ]:
# model = load_model("C:\\Users\\MSIQ\\Downloads\\deployment\\audio_model.h5")

# **Facial Expression**


In [7]:
mp_face_detection = mp.solutions.face_detection

In [8]:
def crop_face_from_frame(frame):
    height, width, _ = frame.shape
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            detection = results.detections[0]
            bboxC = detection.location_data.relative_bounding_box
            x = int(bboxC.xmin * width)
            y = int(bboxC.ymin * height)
            w = int(bboxC.width * width)
            h = int(bboxC.height * height)

            padding = 0.2
            pad_x = int(padding * w)
            pad_y = int(padding * h)
            x = max(0, x - pad_x)
            y = max(0, y - pad_y)
            w = min(width - x, w + 2 * pad_x)
            h = min(height - y, h + 2 * pad_y)

            cropped_face = frame[y:y+h, x:x+w]
            cropped_face_resized = cv2.resize(cropped_face, (width, height))
            return cropped_face_resized
    return None


In [9]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=False,  # to tell the model it is a video not image
        max_num_faces=1,   # number of faces the model should detect
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        refine_landmarks=True  # add points for gaze tracking
    )

In [10]:
def mediapipe_predictions(frame,model):
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = face_mesh.process(frame)
  return results

In [11]:
def extract_frame_points(face_landmarks):
  frame_points=[]
  for landmark in face_landmarks.landmark:
    x = landmark.x
    y = landmark.y
    z = landmark.z
    frame_points.append(x)
    frame_points.append(y)
    frame_points.append(z)

  return frame_points

In [12]:
def apply_facemesh(frame):
  results=mediapipe_predictions(frame,face_mesh)
  if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
      frame_points=extract_frame_points(face_landmarks)
  else:
    frame_points=np.zeros(478*3)
  return frame_points

In [13]:
labels=['truth','lie']

In [14]:


# def make_prediction(input_video):
#     # Open the video capture
#     video_capture = cv2.VideoCapture(input_video)
#     all_video_predictions = []

#     # Check if video file was opened successfully
#     if not video_capture.isOpened():
#         return 'Could not open video'

#     # Validate the input video (ensuring only one person and clear face visibility)
#     valid_input = input_validation(input_video)
#     if not valid_input:
#         return "The input video should have only one person with a clearly visible face"

#     frame_count = 0
#     cur_video32 = []

#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break

#         frame_count += 1

#         # Crop and process face from the current frame
#         cropped_frame = crop_face_from_frame(frame)
#         if cropped_frame is None:
#             cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
#         else:
#             frame_points = apply_facemesh(cropped_frame)  # Get facial landmarks
#             cur_video32.append(frame_points)

#         # Process 32 frames before making a prediction
#         if frame_count == 32:
#             input_data = np.array([cur_video32])  # Create input array for the model
#             if saved_model is not None:
#                 prediction = saved_model.predict(input_data)  # Get the prediction
#                 all_video_predictions.append(np.argmax(prediction,axis=1))
#             else:
#                 return 'Model is not loaded properly.'
#             cur_video32 = []  # Reset for the next 32 frames
#             frame_count = 0

#     # Handle the last frames if less than 32
#     if frame_count != 0:
#         while frame_count < 32:
#             cur_video32.append(np.zeros(478 * 3))  # Zero-padding
#             frame_count += 1
#         input_data = np.array([cur_video32])  # Create input array for the model
#         if saved_model is not None:
#             prediction = saved_model.predict(input_data)
#             all_video_predictions.append(np.argmax(prediction,axis=1))
#         else:
#             return 'Model is not loaded properly.'

#     return all_video_predictions


In [16]:
# x=make_prediction(input_video)

In [17]:
# x

In [18]:
# predictions_combined = np.vstack(x)
# facial_predictions = np.argmax(predictions_combined, axis=1)

In [19]:
# facial_predictions

In [20]:
# def make_prediction(input_video):
#     global saved_model

#     # Open video capture
#     video_capture = cv2.VideoCapture(input_video)
#     all_video_predictions = []

#     if not video_capture.isOpened():
#         return {"error": "Could not open video"}

#     valid_input = input_validation(input_video)
#     if not valid_input:
#         return {"error": "The input video should have only one person with a clearly visible face"}

#     frame_count = 0
#     cur_video32 = []

#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break

#         frame_count += 1

#         cropped_frame = crop_face_from_frame(frame)
#         if cropped_frame is None:
#             cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
#         else:
#             frame_points = apply_facemesh(cropped_frame)
#             cur_video32.append(frame_points)

#         if frame_count == 32:
#             input_data = np.array([cur_video32])
#             prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction



#             all_video_predictions.append(np.argmax(prediction))


#             cur_video32 = []
#             frame_count = 0

#     # Handle remaining frames
#     if frame_count != 0:
#         while frame_count < 32:
#             cur_video32.append(np.zeros(478 * 3))
#             frame_count += 1
#         input_data = np.array([cur_video32])
#         prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction




#         all_video_predictions.append(np.argmax(prediction))


#     return {"predictions": all_video_predictions}

In [21]:
# x=make_prediction(input_video)

In [22]:
# x

In [34]:
# 2

from flask import Flask, request, jsonify
import numpy as np
import os
import cv2

app = Flask(__name__)

# Set upload folder
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

latest_result = None

# Dummy functions (replace these with your actual implementations)
def input_validation(video_path):
    return True  # Placeholder for actual validation logic

def crop_face_from_frame(frame):
    return frame  # Placeholder function

def apply_facemesh(frame):
    return np.random.rand(478 * 3)  # Simulate face mesh extraction

# Load your ML model here
saved_model = None  # Replace with actual model loading

def make_prediction(input_video):
    global saved_model

    # Open video capture
    video_capture = cv2.VideoCapture(input_video)
    all_video_predictions = []

    if not video_capture.isOpened():
        return {"error": "Could not open video"}

    valid_input = input_validation(input_video)
    if not valid_input:
        return {"error": "The input video should have only one person with a clearly visible face"}

    frame_count = 0
    cur_video32 = []

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1

        cropped_frame = crop_face_from_frame(frame)
        if cropped_frame is None:
            cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
        else:
            frame_points = apply_facemesh(cropped_frame)
            cur_video32.append(frame_points)

        if frame_count == 32:
            input_data = np.array([cur_video32])
            prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

            # Convert prediction to a native Python type if needed
            all_video_predictions.append(int(np.argmax(prediction)))  # Convert to int

            cur_video32 = []
            frame_count = 0

    # Handle remaining frames
    if frame_count != 0:
        while frame_count < 32:
            cur_video32.append(np.zeros(478 * 3))
            frame_count += 1
        input_data = np.array([cur_video32])
        prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

        # Convert prediction to a native Python type if needed
        all_video_predictions.append(int(np.argmax(prediction)))  # Convert to int

    return {"predictions": all_video_predictions}


@app.route("/analyze", methods=["POST"])
def analyze():
    global latest_result

    if "video" not in request.files:
        return jsonify({"error": "No video file provided"}), 400

    video_file = request.files["video"]

    if video_file.filename == "":
        return jsonify({"error": "No selected file"}), 400

    file_path = os.path.join(app.config["UPLOAD_FOLDER"], video_file.filename)
    video_file.save(file_path)

    result = make_prediction(file_path)
    latest_result = result  # Store latest result

    return jsonify(result)


@app.route("/result", methods=["GET"])
def get_result():
    if latest_result is None:
        return jsonify({"error": "No analysis has been performed yet"}), 404
    return jsonify(latest_result)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [30]:
# def make_prediction(input_video):
#     global saved_model

#     # Open video capture
#     video_capture = cv2.VideoCapture(input_video)
#     all_video_predictions = []

#     if not video_capture.isOpened():
#         return {"error": "Could not open video"}

#     valid_input = input_validation(input_video)
#     if not valid_input:
#         return {"error": "The input video should have only one person with a clearly visible face"}

#     frame_count = 0
#     cur_video32 = []

#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break

#         frame_count += 1

#         cropped_frame = crop_face_from_frame(frame)
#         if cropped_frame is None:
#             cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
#         else:
#             frame_points = apply_facemesh(cropped_frame)
#             cur_video32.append(frame_points)

#         if frame_count == 32:
#             input_data = np.array([cur_video32])
#             prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

#             # Convert prediction to a native Python type if needed
#             all_video_predictions.append((np.argmax(prediction)))  # Convert to int

#             cur_video32 = []
#             frame_count = 0

#     # Handle remaining frames
#     if frame_count != 0:
#         while frame_count < 32:
#             cur_video32.append(np.zeros(478 * 3))
#             frame_count += 1
#         input_data = np.array([cur_video32])
#         prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

#         # Convert prediction to a native Python type if needed
#         all_video_predictions.append((np.argmax(prediction)))  # Convert to int

#     return {"predictions": all_video_predictions}

In [31]:
# x=make_prediction(input_video)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━

In [32]:
# x

{'predictions': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:

# def make_prediction(input_video):
#     global saved_model

#     # Open video capture
#     video_capture = cv2.VideoCapture(input_video)
#     all_video_predictions = []

#     if not video_capture.isOpened():
#         return {"error": "Could not open video"}

#     valid_input = input_validation(input_video)
#     if not valid_input:
#         return {"error": "The input video should have only one person with a clearly visible face"}

#     frame_count = 0
#     cur_video32 = []

#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break

#         frame_count += 1

#         cropped_frame = crop_face_from_frame(frame)
#         if cropped_frame is None:
#             cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
#         else:
#             frame_points = apply_facemesh(cropped_frame)
#             cur_video32.append(frame_points)

#         if frame_count == 32:
#             input_data = np.array([cur_video32])
#             prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

#             # Convert prediction to a native Python type if needed
#             all_video_predictions.append((np.argmax(prediction)))  # Convert to int

#             cur_video32 = []
#             frame_count = 0

#     # Handle remaining frames
#     if frame_count != 0:
#         while frame_count < 32:
#             cur_video32.append(np.zeros(478 * 3))
#             frame_count += 1
#         input_data = np.array([cur_video32])
#         prediction = saved_model.predict(input_data) if saved_model else [0]  # Dummy prediction

#         # Convert prediction to a native Python type if needed
#         all_video_predictions.append((np.argmax(prediction)))  # Convert to int

#     return {"predictions": all_video_predictions}


In [ ]:
# x=make_prediction(input_video)


In [ ]:
# x

In [ ]:
#1

# from flask import Flask, request, jsonify
# import numpy as np
# import os
# import cv2

# app = Flask(__name__)

# # Set upload folder
# UPLOAD_FOLDER = "uploads"
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# latest_result = None

# # Dummy functions (replace these with your actual implementations)
# def input_validation(video_path):
#     return True  # Placeholder for actual validation logic

# def crop_face_from_frame(frame):
#     return frame  # Placeholder function

# def apply_facemesh(frame):
#     return np.random.rand(478 * 3)  # Simulate face mesh extraction

# # Load your ML model here
# saved_model = None  # Replace with actual model loading

# def make_prediction(input_video):
#     global saved_model

#     # Open video capture
#     video_capture = cv2.VideoCapture(input_video)
#     all_video_predictions = []

#     if not video_capture.isOpened():
#         return {"error": "Could not open video"}

#     valid_input = input_validation(input_video)
#     if not valid_input:
#         return {"error": "The input video should have only one person with a clearly visible face"}

#     frame_count = 0
#     cur_video32 = []

#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break

#         frame_count += 1

#         cropped_frame = crop_face_from_frame(frame)
#         if cropped_frame is None:
#             cur_video32.append(np.zeros(478 * 3))  # Add zeros if no face detected
#         else:
#             frame_points = apply_facemesh(cropped_frame)
#             cur_video32.append(frame_points)

#         if frame_count == 32:
#             input_data = np.array([cur_video32])
#             prediction = saved_model.predict(input_data) if saved_model else [0]


#             all_video_predictions.append(np.argmax(prediction))


#             cur_video32 = []
#             frame_count = 0

#     # Handle remaining frames
#     if frame_count != 0:
#         while frame_count < 32:
#             cur_video32.append(np.zeros(478 * 3))
#             frame_count += 1
#         input_data = np.array([cur_video32])
#         prediction = saved_model.predict(input_data) if saved_model else [0]



#         all_video_predictions.append(np.argmax(prediction))


#     return {"predictions": all_video_predictions}

# @app.route("/analyze", methods=["POST"])
# def analyze():
#     global latest_result

#     if "video" not in request.files:
#         return jsonify({"error": "No video file provided"}), 400

#     video_file = request.files["video"]

#     if video_file.filename == "":
#         return jsonify({"error": "No selected file"}), 400

#     file_path = os.path.join(app.config["UPLOAD_FOLDER"], video_file.filename)
#     video_file.save(file_path)

#     result = make_prediction(file_path)
#     latest_result = result  # Store latest result

#     return jsonify(result)


# @app.route("/result", methods=["GET"])
# def get_result():
#     if latest_result is None:
#         return jsonify({"error": "No analysis has been performed yet"}), 404
#     return jsonify(latest_result)


# if __name__ == "__main__":
#     app.run(host="0.0.0.0", port=5000, debug=True)


# **audio**

In [ ]:
def extract_audio_from_video(video_file, audio_file):
    video = VideoFileClip(video_file)
    audio = video.audio
    audio.write_audiofile(audio_file, codec='pcm_s16le')


In [ ]:
import librosa
import numpy as np

def split_and_pad_audio(audio_file, segment_duration=10, sample_rate=16000):
    segment_samples = segment_duration * sample_rate

    audio, sr = librosa.load(audio_file, sr=sample_rate)

    num_segments = len(audio) // segment_samples
    remainder = len(audio) % segment_samples

    segments = []
    for i in range(num_segments):
        segment = audio[i * segment_samples : (i + 1) * segment_samples]
        segments.append(segment)

    if remainder > 0:
        last_segment = audio[num_segments * segment_samples:]
        padded_segment = np.pad(last_segment, (0, segment_samples - len(last_segment)), mode='constant')
        segments.append(padded_segment)

    return segments


In [ ]:


def extract_mfcc_from_segments(segments, sample_rate=16000, n_mfcc=13):
    mfcc_features = []

    for idx, segment in enumerate(segments):
        mfcc = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)

        feature_entry = {
            "segment_index": idx + 1,
        }
        for i in range(n_mfcc):
            feature_entry[f"mfcc{i+1}_mean"] = mfcc_mean[i]
            feature_entry[f"mfcc{i+1}_std"] = mfcc_std[i]

        mfcc_features.append(feature_entry)

    return pd.DataFrame(mfcc_features)



# **facial expression predictions**

# **audio predictions**

In [ ]:
def make_prediction_audio(input_video):
  video_capture = cv2.VideoCapture(input_video)
  all_video_predictions=[]
  if not video_capture.isOpened():
    return('Could not open video')
  valid_input_voice=check_audio_in_video(input_video)
  if valid_input_voice == False:
     return ("the input video should have audio")
  audio_file = "extracted_audio.wav"
  extract_audio_from_video(input_video, audio_file)
  segments = split_and_pad_audio(audio_file, segment_duration=10, sample_rate=16000)
  mfcc_data_df = extract_mfcc_from_segments(segments, sample_rate=16000, n_mfcc=13)
  mfcc_data_df.drop(['segment_index'], axis=1, inplace=True)
  scaler = load('scaler.joblib')
  mfcc_data_df = scaler.transform(mfcc_data_df)
  mfcc_data_reshaped = np.expand_dims(mfcc_data_df, axis=1)
  y_pred = model.predict(mfcc_data_reshaped)
  voice_predictions = np.argmax(y_pred, axis=1)
  voice_predictions = 1 - voice_predictions
  replicated_voice_predictions = np.repeat(voice_predictions, 10, axis=0)
  return replicated_voice_predictions


In [ ]:
# !pip install flask flask-ngrok

In [ ]:
# !pip install flask-ngrok
# !apt-get install -y ngrok

In [ ]:
# !pip install pyngrok

# **voting**

In [ ]:
# !ngrok authtoken YOUR_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
# import numpy as np

# app = Flask(__name__)
# run_with_ngrok(app)  # This will open an ngrok tunnel

# latest_result = None  # متغير لتخزين النتيجة الأخيرة

# def make_prediction_audio(input_video):
#     return np.random.randint(0, 2, size=(len(input_video) // 10,)).tolist()

# def make_prediction(input_video):
#     return np.random.randint(0, 2, size=(len(input_video), 1)).tolist()

# def final_decision(input_video):
#     global latest_result

#     voice_predictions = make_prediction_audio(input_video)
#     facial_predictions = make_prediction(input_video)

#     predictions_combined = np.vstack(facial_predictions)
#     facial_predictions = np.argmax(predictions_combined, axis=1)

#     replicated_voice_predictions = np.repeat(voice_predictions, 10, axis=0)
#     replicated_voice_predictions = replicated_voice_predictions[:len(facial_predictions)]

#     hard_vote = (np.array(replicated_voice_predictions) + np.array(facial_predictions)) >= 1
#     hard_vote = hard_vote.astype(int)

#     lie = np.sum(hard_vote)
#     truth = len(hard_vote) - lie
#     confidence = (max(lie, truth) / len(hard_vote)) * 100
#     final_decision = "Lie" if lie > truth else "Truth"

#     latest_result = {
#         "voice_predictions": replicated_voice_predictions.tolist(),
#         "facial_predictions": facial_predictions.tolist(),
#         "hard_vote": hard_vote.tolist(),
#         "final_decision": final_decision,
#         "confidence": confidence
#     }
#     return latest_result

# @app.route("/analyze", methods=["POST"])
# def analyze():
#     data = request.json
#     input_video = data.get("video", [])  # استلام بيانات الفيديو كقائمة
#     if not input_video:
#         return jsonify({"error": "No video data provided"}), 400

#     result = final_decision(input_video)
#     return jsonify(result)

# @app.route("/result", methods=["GET"])
# def get_result():
#     if latest_result is None:
#         return jsonify({"error": "No analysis has been performed yet"}), 404
#     return jsonify(latest_result)

# # Run the app with debug mode set in app.config instead of passing it directly to app.run()
# if __name__ == "__main__":
#     app.config["DEBUG"] = True
#     app.run()


In [ ]:
# from flask import Flask, request, jsonify
# import numpy as np

# app = Flask(__name__)

# latest_result = None

# def make_prediction_audio(input_video):
#     return np.random.randint(0, 2, size=(len(input_video) // 10,)).tolist()

# def make_prediction(input_video):
#     return np.random.randint(0, 2, size=(len(input_video), 1)).tolist()

# def final_decision(input_video):
#     global latest_result

#     voice_predictions = make_prediction_audio(input_video)
#     facial_predictions = make_prediction(input_video)

#     predictions_combined = np.vstack(facial_predictions)
#     facial_predictions = np.argmax(predictions_combined, axis=1)

#     replicated_voice_predictions = np.repeat(voice_predictions, 10, axis=0)
#     replicated_voice_predictions = replicated_voice_predictions[:len(facial_predictions)]

#     hard_vote = (np.array(replicated_voice_predictions) + np.array(facial_predictions)) >= 1
#     hard_vote = hard_vote.astype(int)

#     lie = np.sum(hard_vote)
#     truth = len(hard_vote) - lie
#     confidence = (max(lie, truth) / len(hard_vote)) * 100
#     final_decision = "Lie" if lie > truth else "Truth"

#     latest_result = {
#         "voice_predictions": replicated_voice_predictions.tolist(),
#         "facial_predictions": facial_predictions.tolist(),
#         "hard_vote": hard_vote.tolist(),
#         "final_decision": final_decision,
#         "confidence": confidence
#     }
#     return latest_result

# @app.route("/analyze", methods=["POST"])
# def analyze():
#     data = request.files
#     input_video = data.get("video", [])
#     if not input_video:
#         return jsonify({"error": "No video data provided"}), 400

#     result = final_decision(input_video)
#     return jsonify(result)

# @app.route("/result", methods=["GET"])
# def get_result():
#     if latest_result is None:
#         return jsonify({"error": "No analysis has been performed yet"}), 404
#     return jsonify(latest_result)

# if __name__ == "__main__":
#     app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# from flask import Flask, request, jsonify
# import numpy as np
# import os

# app = Flask(__name__)

# # Set upload folder
# UPLOAD_FOLDER = "uploads"
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# latest_result = None

# def make_prediction_audio(input_video):
#     return np.random.randint(0, 2, size=(len(input_video) // 10,)).tolist()

# def make_prediction(input_video):
#     return np.random.randint(0, 2, size=(len(input_video), 1)).tolist()

# def final_decision(video_path):
#     global latest_result

#     # Simulate reading video file as binary
#     with open(video_path, "rb") as f:
#         input_video = f.read()

#     # Fake predictions
#     voice_predictions = make_prediction_audio(input_video)
#     facial_predictions = make_prediction(input_video)

#     predictions_combined = np.vstack(facial_predictions)
#     facial_predictions = np.argmax(predictions_combined, axis=1)

#     replicated_voice_predictions = np.repeat(voice_predictions, 10, axis=0)
#     replicated_voice_predictions = replicated_voice_predictions[:len(facial_predictions)]

#     hard_vote = (np.array(replicated_voice_predictions) + np.array(facial_predictions)) >= 1
#     hard_vote = hard_vote.astype(int)

#     lie = np.sum(hard_vote)
#     truth = len(hard_vote) - lie
#     confidence = (max(lie, truth) / len(hard_vote)) * 100
#     final_decision = "Lie" if lie > truth else "Truth"

#     latest_result = {
#         "voice_predictions": replicated_voice_predictions.tolist(),
#         "facial_predictions": facial_predictions.tolist(),
#         "hard_vote": hard_vote.tolist(),
#         "final_decision": final_decision,
#         "confidence": confidence
#     }
#     return latest_result
# @app.route("/analyze", methods=["POST"])
# def analyze():
#     print("Received request with Content-Type:", request.content_type)
#     print("Request files:", request.files)

#     if "video" not in request.files:
#         return jsonify({"error": "No video file provided"}), 400

#     video_file = request.files["video"]

#     if video_file.filename == "":
#         return jsonify({"error": "No selected file"}), 400

#     file_path = os.path.join(app.config["UPLOAD_FOLDER"], video_file.filename)
#     video_file.save(file_path)

#     result = final_decision(file_path)
#     return jsonify(result)


# @app.route("/result", methods=["GET"])
# def get_result():
#     if latest_result is None:
#         return jsonify({"error": "No analysis has been performed yet"}), 404
#     return jsonify(latest_result)

# if __name__ == "__main__":
#     app.run(host="0.0.0.0", port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# from flask import Flask, request, jsonify
# import numpy as np
# import os

# app = Flask(__name__)

# # Set upload folder
# UPLOAD_FOLDER = "uploads"
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# latest_result = None

# def make_prediction_audio(input_video):
#     return np.random.randint(0, 2, size=(len(input_video) // 10,)).tolist()

# def make_prediction(input_video):
#     return np.random.randint(0, 2, size=(len(input_video), 1)).tolist()

# def final_decision(video_path):
#     global latest_result

#     if not os.path.exists(video_path):
#         return {"error": "File not found"}

#     print(f"Processing file: {video_path}")

#     with open(video_path, "rb") as f:
#         input_video = f.read()

#     if not input_video:
#         return {"error": "File is empty or unreadable"}

#     # Fake predictions
#     voice_predictions = make_prediction_audio(input_video)
#     facial_predictions = make_prediction(input_video)

#     facial_predictions = np.vstack(np.array(facial_predictions))  # Ensure correct format
#     facial_predictions = np.argmax(facial_predictions, axis=1)

#     replicated_voice_predictions = np.repeat(voice_predictions, 10, axis=0)
#     replicated_voice_predictions = replicated_voice_predictions[:len(facial_predictions)]

#     hard_vote = (np.array(replicated_voice_predictions) + np.array(facial_predictions)) >= 1
#     hard_vote = hard_vote.astype(int)

#     lie = np.sum(hard_vote)
#     truth = len(hard_vote) - lie
#     confidence = (max(lie, truth) / len(hard_vote)) * 100
#     final_decision = "Lie" if lie > truth else "Truth"

#     latest_result = {
#         "voice_predictions": replicated_voice_predictions.tolist(),
#         "facial_predictions": facial_predictions.tolist(),
#         "hard_vote": hard_vote.tolist(),
#         "final_decision": final_decision,
#         "confidence": confidence
#     }
#     return latest_result

# @app.route("/analyze", methods=["POST"])
# def analyze():
#     print("Received request with Content-Type:", request.content_type)
#     print("Request files:", request.files)

#     if "video" not in request.files:
#         return jsonify({"error": "No video file provided"}), 400

#     video_file = request.files["video"]

#     if video_file.filename == "":
#         return jsonify({"error": "No selected file"}), 400

#     file_path = os.path.join(app.config["UPLOAD_FOLDER"], video_file.filename)
#     video_file.save(file_path)

#     result = final_decision(file_path)
#     return jsonify(result)

# @app.route("/result", methods=["GET"])
# def get_result():
#     if latest_result is None:
#         return jsonify({"error": "No analysis has been performed yet"}), 404
#     return jsonify(latest_result)

# if __name__ == "__main__":
#     app.run(host="0.0.0.0", port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# !ngrok http 5000


ERROR:  authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.
ERROR:  Your authtoken: YOUR_AUTHTOKEN
ERROR:  Instructions to install your authtoken are on your ngrok dashboard:
ERROR:  https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_105
ERROR:  https://ngrok.com/docs/errors/err_ngrok_105
ERROR:  
